In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from player import Player
from dealer import Dealer
from game import TrucoGame
from environment import TrucoEnvironment
from actions import game_actions, game_actions_list
from agent import Agent
import random
import itertools
import torch as T
import numpy as np

In [2]:
device = T.device("cuda:0" if T.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda', index=0)

In [4]:
num_players = 6

players = [Player(id) for id in range(num_players)]

agents = [Agent(player=player, state_space_dim=339, action_space_dim=game_actions.shape[0], device=device, batch_size=64, target_update_freq=1500, epsilon_end=0.05, epsilon_decay=1000000) for player in players]

random.shuffle(agents)

envs = [TrucoEnvironment([a1.player, a2.player]) for a1, a2 in zip(agents[0::2], agents[1::2])]

In [5]:
def get_env_by_agent(envs, agent):
    for env in envs:
        if agent.player in env.players:
            return env
    return None

def get_agents_by_env(agents, env):
    playing = []
    for agent in agents:
        if agent.player in env.players:
            playing.append(agent)
    return playing[0], playing[1]

In [6]:
for env in envs:
    
    playing_agents = get_agents_by_env(agents, env)
    
    for agent in playing_agents:

        # reset env
        starting_player, legal_actions, game_state = env.reset()

        # Initialize the ReplayBuffer
        while len(agent.replay_buffer) < agent.min_replay_size:
            # pick legal action given uniform distribution
            action = np.random.choice(legal_actions, 1, [1/len(legal_actions) for i in legal_actions])
            action = game_actions_list.index(action)

            # Take action, observer outcome
            rew, done, next_player, next_legal_actions, new_game_state = env.step(starting_player, action)

            #Save transition for training later
            transition = (game_state, action, rew, done, new_game_state)
            agent.save_transition(transition)

            starting_player = next_player
            legal_actions = next_legal_actions
            game_state = new_game_state

            if done:
                starting_player, legal_actions, game_state = env.reset()

05-Oct-21 17:31:26 - WARNING - Game ended with no winner.
05-Oct-21 17:31:26 - WARNING - Game ended with no winner.
05-Oct-21 17:31:26 - WARNING - Game ended with no winner.
05-Oct-21 17:31:26 - WARNING - Game ended with no winner.
05-Oct-21 17:31:26 - WARNING - Game ended with no winner.
05-Oct-21 17:31:27 - WARNING - Game ended with no winner.


In [7]:
PLAYER_SWAP_FREQ=50000

In [ ]:
for epoch in itertools.count():
    print(f"Starting epoch: {epoch}")
    
    for env in envs:

        starting_player, legal_actions, game_state = env.reset()
        playing_agents = get_agents_by_env(agents, env)

        # Main training loop
        for step in itertools.count():
            agent = [agent for agent in playing_agents if agent.player == starting_player][0]

            action = agent.choose_action(legal_actions, game_state)

            # Take action, observe outcome
            rew, done, next_player, next_legal_actions, new_game_state = env.step(agent.player, action)

            #Save transition for training later
            transition = (game_state, action, rew, done, new_game_state)
            agent.save_transition(transition)

            starting_player = next_player
            legal_actions = next_legal_actions
            game_state = new_game_state

            if done:
                # Give agents their episode reward
                for agent in playing_agents:
                    agent.save_reward(env.game.get_score(agent.player))

                starting_player, legal_actions, game_state = env.reset()

            # Learn from replay buffer
            agent.learn()

            if step % agent.save_freq == 0 and step > 0:
                for agent in playing_agents:
                    T.save(agent.online_net.state_dict(), f"./model_saves/model-v4-{agent.player.get_id()}.pt")
                    print(f"Model saved at step: {step} \r")


            # Logging
            if step % 25000 == 0 and step > 0:
                for agent in playing_agents:
                    avg_reward = np.mean(agent.reward_buffer)
                    print(f"Step: {step} | Player: {agent.player.get_id()} | Avg reward: {avg_reward} | W/L: {max([w if p == agent.player else 0 for p, w in env.games_won])/env.games_played} | Games: {env.games_played}")   

            # Break out of training loop when swap rate is reached
            if step % PLAYER_SWAP_FREQ == 0 and step > 0:
                print(f"Player {playing_agents[0].player.get_id()} vs Player {playing_agents[1].player.get_id()} finished.")
                break;


    # Clear GPU cache
    T.cuda.empty_cache()
                
    # Destroy all envs
    envs.clear()

    random.shuffle(agents)

    # Create new Envs
    envs = [TrucoEnvironment([a1.player, a2.player]) for a1, a2 in zip(agents[0::2], agents[1::2])]
    

05-Oct-21 17:31:27 - WARNING - Game ended with no winner.


Starting epoch: 0
Step: 25000 | Player: 3 | Avg reward: 1.01 | W/L: 0.5226781857451404 | Games: 7408
Step: 25000 | Player: 0 | Avg reward: 0.77 | W/L: 0.4773218142548596 | Games: 7408


05-Oct-21 17:33:36 - WARNING - Game ended with no winner.


Step: 50000 | Player: 3 | Avg reward: 1.22 | W/L: 0.5219945450730821 | Games: 14299
Step: 50000 | Player: 0 | Avg reward: 0.85 | W/L: 0.478005454926918 | Games: 14299
Player 3 vs Player 3 finished.
Step: 25000 | Player: 4 | Avg reward: 0.95 | W/L: 0.5254965110037574 | Games: 7452
Step: 25000 | Player: 2 | Avg reward: 1.07 | W/L: 0.4745034889962426 | Games: 7452
